## Imports 

In [1]:
from statistics import mean
import numpy as np
import pandas as pd
import math
import os
from collections import Counter
from functools import reduce
import glob
import copy

## Opening the CSV files 

In [2]:
dataframes = [pd.read_csv(file, sep=',', index_col=0) for file in sorted(glob.glob('../preprocessed_datasets' + "/*."+'csv'))]
cohorts = [file.strip(".csv") for file in sorted(os.listdir('../preprocessed_datasets'))]

In [3]:
# reduce to BL visit only
all_cohorts = dict()
for name, df in zip(cohorts, dataframes):
    all_cohorts[name] = df.loc[(df["Visit"] == 1) & (df["Diagnosis"].astype(str) == 'MCI')]

## Functions to perform essential calculations 

In [4]:
def cat_stat_df(dfs, result):
    """Counting different categories, calculate the % of categorical features, store results in a df"""
    
    categorical = {'APOE4': [2.0, 1.0], 'Sex': ['Female'], 'Diagnosis': ['CU', 'MCI', 'AD']}
    column_cat = ['Sex', 'Diagnosis', 'APOE4']

    for cohort in dfs:
        
        if dfs[cohort].empty==True:
            continue
        
        else:
            calc_dict = dict()
            df = dfs[cohort]

            for col in column_cat:
                ca = Counter(df[col].dropna())
                calc_dict[col] = ca

            cohort_df = pd.DataFrame(calc_dict).transpose()
            cohort_df = cohort_df.dropna(how='all')
            cohort_df.loc[cohort] = cohort_df.sum()
           
            for i in categorical:
                
                if i == 'Diagnosis':
                    
                    if i in cohort_df.index: 
                        result.loc[cohort, categorical[i]] = cohort_df.loc[cohort, cohort_df.loc[i].notna()].astype(int)
                        result.loc[cohort, categorical[i]] = result.loc[cohort, categorical[i]].replace({np.nan: 0})
                        result.loc[cohort, 'n'] = int(sum(cohort_df.loc[cohort, cohort_df.loc[i].notna()]))
                        result.loc[cohort, 'Total'] = int(len(dfs[cohort].index))
                    
                    else:
                        result.loc[cohort, i] = np.nan
                        result.loc[cohort, 'n'] = int(len(dfs[cohort].index))
                
                elif i == 'APOE4':
                    
                    if 'APOE4' in list(cohort_df.index.astype(str)):
                        
                        if '2.0' not in list(cohort_df.columns.astype(str)) and '2' not in list(cohort_df.columns.astype(str)):
                            cohort_df[2.0] = np.nan
                        
                        result.loc[cohort, i] = round(100 * sum([val for val in cohort_df.loc[i, categorical[i]]]) / 
                                                     sum([val for val in cohort_df.loc[i].dropna()]), 1)
                    
                    else:
                        result.loc[cohort, i] = np.nan
                
                elif i == 'Sex':
                    
                    if (i in cohort_df.index) & ("Female" in cohort_df.columns):
                        result.loc[cohort, i] = round(100 * sum([val for val in cohort_df.loc[i, categorical[i]]]) 
                                                      / sum([val for val in cohort_df.loc[i].dropna()]), 1)
                    else:
                        result.loc[cohort, i] = 0
    
                    
    result.rename(columns={"Sex": "Female %", "APOE4": "APOE4 %"}, inplace=True)
              
    return result

In [5]:
def num_stat_df(dfs, result_df):
    """Calculating std and mean and storing it in the result dataframe"""
    
    column_names = ['Age', 'CDR', 'Education', 'MMSE', 'CDRSB', 'Hippocampus', 'A-beta', 'Ttau', 'Ptau']
    
    for df in dfs:
        dataset = dfs[df]
        calc_dict = dict()
        
        for col in column_names:
            
            if (col in dataset.columns) and (dataset[col].notna().any()):
                df_std = round(np.nanstd(dataset[col]), 1)
                df_mean = round(np.nanmean(dataset[col]), 1)
                dict_value = str(df_mean) + ' (' + str(df_std) + ')'
                calc_dict[col] = dict_value
                
            else:
                calc_dict[col] = np.nan
   
        for key in calc_dict:
            result_df.loc[df, key] = calc_dict[key]
        
    return result_df

## Make an empty dataframe to fill in with the results

In [6]:
results = pd.DataFrame(index = all_cohorts.keys(), columns = [col for col in all_cohorts['AIBL'].columns])
results.index.name = 'Name of Dataset'

for i in ['CU', 'MCI', 'AD', 'Total']:
    results[i] = np.nan

cat_stat_df(all_cohorts, results)
num_stat_df(all_cohorts, results)

results.drop(columns=['Diagnosis', 'Visit', 'Race', 'Months'], inplace=True)
results

,Age,Female %,Education,APOE4 %,CDR,MMSE,CDRSB,Hippocampus,Ttau,Ptau,A-beta,CU,MCI,AD,Total,n
Name of Dataset,,,,,,,,,,,,,,,,
A4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABVIB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADNI,72.9 (7.6),40.9,16.0 (2.8),50.0,0.5 (0.0),27.6 (1.8),1.5 (0.9),6778.3 (1132.2),287.1 (134.7),27.8 (15.0),959.6 (435.3),0.0,1016.0,0.0,1016.0,1016.0
AIBL,76.9 (7.5),55.2,11.8 (3.0),53.0,0.5 (0.3),25.4 (3.6),1.9 (1.7),2.6 (0.4),536.6 (0.0),89.6 (0.0),1046.6 (0.0),0.0,134.0,0.0,134.0,134.0
ANM,76.0 (6.5),54.7,10.0 (4.3),39.6,0.5 (0.1),26.8 (2.0),1.5 (1.0),6349.2 (1131.9),NaN,NaN,NaN,0.0,397.0,0.0,397.0,397.0
ARWIBO,70.5 (7.6),63.0,8.4 (4.2),41.8,0.5 (0.2),26.7 (2.1),NaN,6607.5 (1245.4),464.2 (270.6),79.1 (35.5),501.7 (223.9),0.0,208.0,0.0,208.0,208.0
DOD-ADNI,70.5 (4.0),0,15.6 (2.4),NaN,0.4 (0.2),27.1 (2.2),1.4 (1.0),7395.8 (1672.6),250.5 (99.6),22.8 (10.2),1317.4 (592.6),0.0,27.0,0.0,27.0,27.0
EDSD,70.9 (7.3),46.4,11.7 (2.9),52.6,NaN,26.5 (2.1),NaN,6869.6 (1151.6),442.2 (266.8),80.0 (39.7),665.5 (348.0),0.0,140.0,0.0,140.0,140.0
EMIF,69.7 (7.8),47.0,10.7 (3.9),NaN,0.5 (0.1),26.3 (2.6),NaN,6816.9 (1235.4),414.4 (305.2),64.4 (33.6),624.5 (298.8),0.0,526.0,0.0,526.0,526.0


## Final table 

In [7]:
results[['n', 'Total', 'CU', 'MCI', 'AD', 'Female %', 'Age', 'Education', 'MMSE', 'CDR', 'CDRSB', 'APOE4 %', 'Hippocampus']]

,n,Total,CU,MCI,AD,Female %,Age,Education,MMSE,CDR,CDRSB,APOE4 %,Hippocampus
Name of Dataset,,,,,,,,,,,,,
A4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABVIB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADNI,1016.0,1016.0,0.0,1016.0,0.0,40.9,72.9 (7.6),16.0 (2.8),27.6 (1.8),0.5 (0.0),1.5 (0.9),50.0,6778.3 (1132.2)
AIBL,134.0,134.0,0.0,134.0,0.0,55.2,76.9 (7.5),11.8 (3.0),25.4 (3.6),0.5 (0.3),1.9 (1.7),53.0,2.6 (0.4)
ANM,397.0,397.0,0.0,397.0,0.0,54.7,76.0 (6.5),10.0 (4.3),26.8 (2.0),0.5 (0.1),1.5 (1.0),39.6,6349.2 (1131.9)
ARWIBO,208.0,208.0,0.0,208.0,0.0,63.0,70.5 (7.6),8.4 (4.2),26.7 (2.1),0.5 (0.2),NaN,41.8,6607.5 (1245.4)
DOD-ADNI,27.0,27.0,0.0,27.0,0.0,0,70.5 (4.0),15.6 (2.4),27.1 (2.2),0.4 (0.2),1.4 (1.0),NaN,7395.8 (1672.6)
EDSD,140.0,140.0,0.0,140.0,0.0,46.4,70.9 (7.3),11.7 (2.9),26.5 (2.1),NaN,NaN,52.6,6869.6 (1151.6)
EMIF,526.0,526.0,0.0,526.0,0.0,47.0,69.7 (7.8),10.7 (3.9),26.3 (2.6),0.5 (0.1),NaN,NaN,6816.9 (1235.4)
